In [1]:
import pandas as pd
import numpy as np

import seaborn as sns

import matplotlib.pyplot as plt

from scipy import stats

from sklearn.model_selection import cross_val_score
from sklearn.cross_validation import KFold

from sklearn.preprocessing import StandardScaler

from scipy import stats

%matplotlib inline

I:\Anaconda3\lib\site-packages\sklearn\cross_validation.py:44: DeprecationWarning: This module was deprecated in version 0.18 in favor of the model_selection module into which all the refactored classes and functions are moved. Also note that the interface of the new CV iterators are different from that of this module. This module will be removed in 0.20.
  "This module will be removed in 0.20.", DeprecationWarning)


In [504]:
train_data = pd.read_csv('train.csv')

In [505]:
#+++++++++++++++
#+++++++++++++++
test_data = pd.read_csv('test.csv')
test_id = test_data.Id
test_data = test_data.drop('Id' , axis = 1)

In [506]:
train_data = train_data.drop(['Id'] , axis = 1)

In [507]:
numerical_columns = train_data.dtypes[train_data.dtypes != object].index #含有预测值 SalePrice
categorical_columns = train_data.dtypes[train_data.dtypes == object].index

In [508]:
missing_columns_num = (train_data[numerical_columns].isnull().sum()[train_data[numerical_columns].isnull().sum().values>0]).index
#含有缺失值得numerical columns

In [509]:
missing_columns_num#含有确实值得numerical的columns 缺失值处理
#先不处理这些缺失值 暂时不处理
#后续可能会修补
#=======================
#=======================
#=======================

Index(['LotFrontage', 'MasVnrArea', 'GarageYrBlt'], dtype='object')

In [510]:
#+++++++++++++++++++
#+++++++++++++++++++
#test中含有缺失值的numerical features
test_numerical_columns = list(numerical_columns)
test_numerical_columns.remove('SalePrice')
test_missing_columns_num = (test_data[test_numerical_columns].isnull().sum()[test_data[test_numerical_columns].isnull().sum().values>0]).index
test_missing_columns_num

Index(['LotFrontage', 'MasVnrArea', 'BsmtFinSF1', 'BsmtFinSF2', 'BsmtUnfSF',
       'TotalBsmtSF', 'BsmtFullBath', 'BsmtHalfBath', 'GarageYrBlt',
       'GarageCars', 'GarageArea'],
      dtype='object')

In [511]:
#++++++++++++++++++
#++++++++++++++++++
#只需要对下面的含有缺失值的numerical features进行填充 test中别的numerical features在model中用不上
set(test_missing_columns_num)&set(largest_10_corr_matrix_index)

{'GarageCars', 'TotalBsmtSF'}

In [512]:
#++++++++++++++++
#++++++++++++++++
#使用的numerical features进行缺失值填充
test_data.GarageCars = test_data.GarageCars.fillna(2)
test_data.TotalBsmtSF = test_data.TotalBsmtSF.fillna(test_data.TotalBsmtSF.mean())

In [513]:
corr_matrix = train_data.corr() #数字型feature的皮尔逊相关系数

In [514]:
largest_10_corr_matrix = corr_matrix.nlargest(10 , 'SalePrice')
largest_10_corr_matrix_index = largest_10_corr_matrix.index #largest corr
largest = train_data[largest_10_corr_matrix_index].corr()

largest_10_corr_matrix_index = list(set(largest_10_corr_matrix_index)-set(['GarageArea' , '1stFlrSF'])) #已经存在与其相关的的feature 去掉

In [401]:
#==========
#==========
#==========
#下面是处理numerical的离群值

In [515]:
train_data.loc[train_data.TotalBsmtSF>6000].index #根据上图找离群值
train_data = train_data.drop(1298) #删除离群值

In [516]:
train_data.loc[train_data.GrLivArea>4500,'GrLivArea'].index #找离群点
train_data = train_data.drop([523]) #上面已经把1298删除了
#注意删除前面的离群值后 后面的样本序号会变

In [517]:
#进行对数除偏
train_data.SalePrice = train_data.SalePrice.apply(np.log)
train_data.GrLivArea = train_data.GrLivArea.apply(np.log)

In [518]:
#+++++++++++++++
#+++++++++++++++
test_data.GrLivArea = test_data.GrLivArea.apply(np.log)

In [519]:
train_data['yearcat'] = pd.Series( 99 , index = train_data.YearBuilt.index)
#测试8分组
train_data.loc[ (train_data.YearBuilt>1871.999) & (train_data.YearBuilt<=1929.0)  , 'yearcat'] = 1
train_data.loc[ (train_data.YearBuilt>1929.0) & (train_data.YearBuilt<=1954.0)  , 'yearcat'] = 2
train_data.loc[ (train_data.YearBuilt>1954.0) & (train_data.YearBuilt<=1963.0)  , 'yearcat'] = 3
train_data.loc[ (train_data.YearBuilt>1963.0) & (train_data.YearBuilt<=1972.5)  , 'yearcat'] = 4
train_data.loc[ (train_data.YearBuilt>1972.5) & (train_data.YearBuilt<=1990.0)  , 'yearcat'] = 5
train_data.loc[ (train_data.YearBuilt>1990.0) & (train_data.YearBuilt<=2000.0)  , 'yearcat'] = 6
train_data.loc[ (train_data.YearBuilt>2000.0) & (train_data.YearBuilt<=2005.0)  , 'yearcat'] = 7
train_data.loc[ (train_data.YearBuilt>2005.0) & (train_data.YearBuilt<=2010.0)  , 'yearcat'] = 8

train_data.YearBuilt = train_data.yearcat.copy()
train_data = train_data.drop('yearcat' , axis = 1) #删除辅助feature

In [520]:
#+++++++++++++++++++
#+++++++++++++++++++
test_data['yearcat'] = pd.Series( 99 , index = test_data.YearBuilt.index)
#测试8分组
test_data.loc[ (test_data.YearBuilt>1871.999) & (test_data.YearBuilt<=1929.0)  , 'yearcat'] = 1
test_data.loc[ (test_data.YearBuilt>1929.0) & (test_data.YearBuilt<=1954.0)  , 'yearcat'] = 2
test_data.loc[ (test_data.YearBuilt>1954.0) & (test_data.YearBuilt<=1963.0)  , 'yearcat'] = 3
test_data.loc[ (test_data.YearBuilt>1963.0) & (test_data.YearBuilt<=1972.5)  , 'yearcat'] = 4
test_data.loc[ (test_data.YearBuilt>1972.5) & (test_data.YearBuilt<=1990.0)  , 'yearcat'] = 5
test_data.loc[ (test_data.YearBuilt>1990.0) & (test_data.YearBuilt<=2000.0)  , 'yearcat'] = 6
test_data.loc[ (test_data.YearBuilt>2000.0) & (test_data.YearBuilt<=2005.0)  , 'yearcat'] = 7
test_data.loc[ (test_data.YearBuilt>2005.0) & (test_data.YearBuilt<=2010.0)  , 'yearcat'] = 8

test_data.YearBuilt = test_data.yearcat.copy()
test_data = test_data.drop('yearcat' , axis = 1) #删除辅助feature

In [521]:
print(train_data.shape , test_data.shape)

(1458, 80) (1459, 79)


### 后面进行处理categorical features

In [522]:
#++++++++++++++++++
#++++++++++++++++++
test_missing_columns_cat = categorical_columns[test_data[categorical_columns].isnull().sum().values != 0]
test_missing_columns_cat

Index(['MSZoning', 'Alley', 'Utilities', 'Exterior1st', 'Exterior2nd',
       'MasVnrType', 'BsmtQual', 'BsmtCond', 'BsmtExposure', 'BsmtFinType1',
       'BsmtFinType2', 'KitchenQual', 'Functional', 'FireplaceQu',
       'GarageType', 'GarageFinish', 'GarageQual', 'GarageCond', 'PoolQC',
       'Fence', 'MiscFeature', 'SaleType'],
      dtype='object')

In [523]:
#有缺失值的categorical features
missing_columns_cat = categorical_columns[train_data[categorical_columns].isnull().sum().values != 0]
missing_columns_cat

Index(['Alley', 'MasVnrType', 'BsmtQual', 'BsmtCond', 'BsmtExposure',
       'BsmtFinType1', 'BsmtFinType2', 'Electrical', 'FireplaceQu',
       'GarageType', 'GarageFinish', 'GarageQual', 'GarageCond', 'PoolQC',
       'Fence', 'MiscFeature'],
      dtype='object')

In [524]:
missing_columns_cat_most = ['Alley' , 'FireplaceQu' , 'PoolQC' , 'Fence' , 'MiscFeature'] #这几个feature 不适用 缺失太多可以直接删除
#fireplacequ 贡献太小
#缺失值太多的features
train_data = train_data.drop(missing_columns_cat_most , axis = 1) #删除了 以前忘记删除了

In [525]:
#++++++++++++++++++++
#++++++++++++++++++++
test_data = test_data.drop(missing_columns_cat_most , axis = 1)

In [526]:
#++++++++++++++++++++++
#++++++++++++++++++++++
test_missing_columns_cat_more = list(set(test_missing_columns_cat) - set(missing_columns_cat_most))
test_missing_columns_cat_more

['Functional',
 'GarageType',
 'Exterior2nd',
 'GarageQual',
 'GarageFinish',
 'BsmtFinType1',
 'MasVnrType',
 'SaleType',
 'MSZoning',
 'Exterior1st',
 'Utilities',
 'BsmtFinType2',
 'BsmtQual',
 'BsmtExposure',
 'BsmtCond',
 'KitchenQual',
 'GarageCond']

In [527]:
#++++++++++++++++++++++
#++++++++++++++++++++++
for i in test_missing_columns_cat_more:
    test_data[i].fillna(test_data[i].describe().top)

In [528]:
missing_columns_cat_more = list(set(missing_columns_cat) - set(missing_columns_cat_most))
missing_columns_cat_more #缺失值不是太多的

['Electrical',
 'GarageQual',
 'GarageFinish',
 'BsmtFinType1',
 'MasVnrType',
 'BsmtQual',
 'BsmtFinType2',
 'GarageCond',
 'BsmtExposure',
 'GarageType',
 'BsmtCond']

In [529]:
#进行填充 直接使用频数最高的进行填充
for i in missing_columns_cat_more:
    train_data[i].fillna(train_data[i].describe().top)


In [530]:
#没有缺失值的categorical features
no_missing_columns_cat = list(set(categorical_columns) - set(missing_columns_cat)) #categorical features 减去含有缺失值的就是 没有缺失值的categorical features
no_missing_columns_cat #这个不予处理 直接转换为哑变量

['Exterior2nd',
 'Heating',
 'Condition2',
 'SaleType',
 'PavedDrive',
 'ExterQual',
 'CentralAir',
 'LandSlope',
 'Functional',
 'HouseStyle',
 'LandContour',
 'RoofStyle',
 'RoofMatl',
 'Exterior1st',
 'Utilities',
 'Neighborhood',
 'LotConfig',
 'KitchenQual',
 'Condition1',
 'MSZoning',
 'LotShape',
 'Street',
 'HeatingQC',
 'Foundation',
 'ExterCond',
 'SaleCondition',
 'BldgType']

In [531]:
all_columns = train_data.columns #保存所有的columns
train_data = pd.get_dummies(train_data)

dummy_columns = list(set(train_data.columns) - set(all_columns)) #获取新增加的哑变量的columns


In [533]:
#+++++++++++++++++
#+++++++++++++++++
test_data = pd.get_dummies(test_data)

In [534]:
test_no_dummy_columns = list(set(train_data.columns) - set(test_data.columns))
test_no_dummy_columns

['RoofMatl_Roll',
 'HouseStyle_2.5Fin',
 'SalePrice',
 'Electrical_Mix',
 'GarageQual_Ex',
 'Exterior2nd_Other',
 'Heating_OthW',
 'Condition2_RRAn',
 'RoofMatl_Metal',
 'Heating_Floor',
 'Exterior1st_ImStucc',
 'Condition2_RRNn',
 'RoofMatl_Membran',
 'Utilities_NoSeWa',
 'Exterior1st_Stone',
 'Condition2_RRAe']

In [535]:
for i in test_no_dummy_columns:
    test_data[i] = 0
    
test_data = test_data.drop('SalePrice' , axis = 1) #删除这个赋值过来的feature

In [538]:
print(train_data.shape , test_data.shape)


(1458, 270) (1459, 269)


In [581]:
from sklearn.linear_model import LinearRegression
from sklearn.svm import SVR
from sklearn.tree import DecisionTreeRegressor
from sklearn.neighbors import KNeighborsRegressor
from sklearn.ensemble import RandomForestRegressor
from sklearn.linear_model import Ridge
from sklearn.linear_model import Lasso

forest = RandomForestRegressor(n_estimators=100)
linear = LinearRegression()
svr = SVR()
dec_tree = DecisionTreeRegressor()
knn = KNeighborsRegressor()
ridge = Ridge(alpha=5.5 , random_state=True)
lasso = Lasso()

classifier = {'random_forest':forest , 'LR':linear , 'SVM':svr , 'decison_tree':dec_tree , 'KNN':knn , 'ridge':ridge , 'lasso':lasso}
#kf = KFold(train_data.shape[0] , 10 , True)

In [51]:
def rmse(model , data , label):
    #cv=5 分为5folds
    #在这里会对模型进行训练 
    return np.sqrt(-cross_val_score(model, data, label, scoring="neg_mean_squared_error", cv = 10))

In [539]:
vali_index = largest_10_corr_matrix_index.copy()

In [540]:
vali_index.remove('SalePrice')

'''numerical features'''
#numerical features
vali_index.append('YearRemodAdd') #这个保留 能减小一点 rmse


#下面两个可以不考虑 已经和别的特征相关了
#vali_index.append('GarageArea')
#vali_index.append('1stFlrSF')
'''下面是categorical features'''

#categorical features 有缺失值的 已经填充 较高相关的
#vali_index.append('BsmtQual')
#vali_index.append('GarageFinish')

#categorical features 没有缺失值 较高相关的
#vali_index.append('KitchenQual')
#vali_index.append('ExterQual')

'下面是categorical features'

In [306]:
#因为下面使用的 哑变量 这里先不使用原有的categorical features
#vali_index =vali_index + no_missing_columns_cat #没有缺失值的categorical features
#vali_index = vali_index + missing_columns_cat_more#有缺失值的categorical features

In [541]:
vali_index+=dummy_columns

In [573]:
#vali_index.remove('TotRmsAbvGrd') #暂时保留这个feature

In [543]:
best_model = _
smallest_rmse = 1.0
for model_name , model in classifier.items():
    result = np.average(rmse(model , train_data[vali_index] , train_data.SalePrice))
    if result<smallest_rmse:
        smallest_rmse = result
        best_model = model_name
        
    print(model_name , result)

print('\nbest:',best_model , smallest_rmse)
#真大

random_forest 0.14054763109632312
LR 0.1266955990743491
SVM 0.2553220667415018
decison_tree 0.200873333429622
KNN 0.26218173013387003
ridge 0.12152991920302023
lasso 0.2663005320945702

best: ridge 0.12152991920302023


In [582]:
np.average(rmse(ridge , train_data[vali_index] , train_data.SalePrice))

0.12136560555577065

In [550]:
from sklearn.model_selection import GridSearchCV

def grid_search_scoring(model , data , label):
    return np.average(rmse(model , data , label))

In [551]:
param_grid={'alpha':[0.5 , 1.0 , 1.5 , 2.0 , 2.5 , 3.0 , 3.5 , 4.0]}
kfold = KFold(train_data.shape[0] , 10 , True)
grid = GridSearchCV(estimator=ridge , param_grid=param_grid , scoring=grid_search_scoring , cv=kfold)

In [552]:
grid.fit(train_data[vali_index] , train_data.SalePrice)

GridSearchCV(cv=sklearn.cross_validation.KFold(n=1458, n_folds=10, shuffle=True, random_state=None),
       error_score='raise',
       estimator=Ridge(alpha=3, copy_X=True, fit_intercept=True, max_iter=None,
   normalize=False, random_state=None, solver='auto', tol=0.001),
       fit_params={}, iid=True, n_jobs=1,
       param_grid={'alpha': [0.5, 1.0, 1.5, 2.0, 2.5, 3.0, 3.5, 4.0]},
       pre_dispatch='2*n_jobs', refit=True, return_train_score=True,
       scoring=<function grid_search_scoring at 0x000001C01F447598>,
       verbose=0)

In [554]:
grid.best_params_ ridge()

{'alpha': 0.5}

In [574]:
#进行预测
#生成结果
ridge.fit(train_data[vali_index] , train_data.SalePrice) #用全部数据训练模型

result = ridge.predict(test_data[vali_index])
result = np.exp(result)
result_csv = pd.DataFrame({'Id':test_id , 'SalePrice':result})
result_csv.to_csv('submit3.csv' , index=False)